In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import folium

CENTER_BARCELONA = {"lat": 41.3951, "lon": 2.1734}

def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df['geometry'] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis='columns'), geometry='geometry')

In [ ]:
DATA_PATH = "../../../data/"

gdfs = [
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_no2_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "NO2"})
    .astype({"NO2": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm2-5_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM2_5"})
    .astype({"PM2_5": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm10_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM10"})
    .astype({"PM10": "category"}),
]

gdfs = [convert_wkt_to_geometry(gdf, "GEOM_WKT") for gdf in gdfs]
gdf = gdfs[0][["TRAM", "geometry"]]
for temp_gdf in gdfs:
    gdf = gdf.merge(temp_gdf.drop(columns=["geometry"]), on="TRAM")

gdf["NO2"] = gdf["NO2"].cat.reorder_categories(
    [
        "10-20 µg/m³",
        "20-30 µg/m³",
        "30-40 µg/m³",
        "40-50 µg/m³",
        "50-60 µg/m³",
        "60-70 µg/m³",
        ">70 µg/m³",
    ],
    ordered=True,
)

gdf["PM2_5"] = gdf["PM2_5"].cat.reorder_categories(
    ["5-10 µg/m³", "10-15 µg/m³", "15-20 µg/m³", "20-25 µg/m³", "25-30 µg/m³"],
    ordered=True,
)

gdf["PM10"] = gdf["PM10"].cat.reorder_categories(
    [
        "<=15 µg/m³",
        "15-20 µg/m³",
        "20-25 µg/m³",
        "25-30 µg/m³",
        "30-35 µg/m³",
        "35-40 µg/m³",
        "> 40 µg/m³",
    ],
    ordered=True,
)

gdf: gpd.GeoDataFrame = gdf.set_crs(epsg=25831).to_crs(epsg=4326)

In [ ]:
map = folium.Map(
    location=list(CENTER_BARCELONA.values()), tiles="CartoDB Positron", zoom_start=12.3
)

polluant = "PM2_5"
# Define a color map for NO2 levels
color_map = dict(zip(gdf[polluant].cat.categories, px.colors.sequential.Plasma_r))

def style_function(feature):
    return {
        'fillColor': color_map.get(feature['properties'][polluant], 'gray'),
        'color': color_map.get(feature['properties'][polluant], 'gray'),
        'weight': 2,
        'fillOpacity': 0.6,
    }

folium.GeoJson(
    gdf.to_json(),
    name="Air Quality",
    tooltip=folium.GeoJsonTooltip(fields=[polluant], aliases=[polluant.replace("_", ".")]),
    style_function=style_function
).add_to(map)

# Add legend
legend_html = f'''
<div style="position: fixed; 
            top: 50px; right: 50px; width: 120px; height: {35 + len(gdf[polluant].cat.categories) * 20}px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 5px 10px;
            ">
    <b>{polluant.replace("_", ".")}</b><br>
    {'<br>'.join(f'<i style="background:{color_map[cat]}">&nbsp;&nbsp;&nbsp;&nbsp;</i> {cat}' for cat in gdf[polluant].cat.categories)}<br>
</div>
'''

map.get_root().html.add_child(folium.Element(legend_html))

# map.save(f"{polluant}_map.html")

map